In [1]:
# Step 1:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

def extract_text_from_epub(epub_path):
    book = epub.read_epub(epub_path)
    text = ""
    
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.content, 'html.parser')
            text += soup.get_text() + "\n\n"
    
    return text

# Example Usage
epub_path = "/Users/sparshpatel/Documents/Codes/codes/pythonml/slm/Percy Jackson and the Olympians 5 - The Last Olympian by Rick Riordan (z-lib.org).epub"  # Replace with your actual file
book_text = extract_text_from_epub(epub_path)
print(book_text[:1000])  # Print a sample


/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):





index








Contents 
Chapter One ....................................................................................................................................... 1 
Chapter Two ........................................................................................................................................ 8 
Chapter Three ................................................................................................................................... 14 
Chapter Four ..................................................................................................................................... 21 
Chapter Five ..................................................................................................................................... 26 
Chapter Six ....................................................................................................................................... 30 
Chapter Seven ....................................................

In [2]:
# Step 2:

import nltk
from nltk.tokenize import sent_tokenize

# Download necessary NLTK data
nltk.download('punkt')

def split_into_chunks(text, max_chunk_size=1024):
    sentences = sent_tokenize(text)  # Split into sentences
    chunks = []
    chunk = ""

    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_chunk_size:
            chunk += sentence + " "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + " "

    if chunk:
        chunks.append(chunk.strip())

    return chunks

# Apply chunking
chunks = split_into_chunks(book_text)
print(f"Total chunks: {len(chunks)}")
print(chunks[10:15])  # Print first 3 chunks for verification


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sparshpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total chunks: 514
['Beckendorf chuckled, and together we soared out over the Atlantic. It was almost dark by the time we spotted our target. The  Princess Andromeda  glowed on \n3 \nthe horizon—a huge cruise ship lit up yellow and white. From a distance, you\'d think it was just a \n3\nparty ship, not the headquarters for the Titan lord. Then as you got closer, you might notice the giant \nfigurehead—a dark-haired maiden in a Greek chiton, wrapped m chains with a look of horror on her \nface, as if she could smell the stench of all the monsters she was being forced to carry. Seeing the ship again twisted my gut into knots. I\'d almost died twice on the  Princess \nAndromeda. Now it was heading straight for New York. "You know what to do?" Beckendorf yelled over the wind. I nodded. We\'d done dry runs at the dockyards in New Jersey, using abandoned ships as \nour targets. I knew how little time we would have. But I also knew this was our best chance to end \nKronos\'s invasion before it

In [2]:
# Another way to categorise chunks, may lead to different answers
# import re
# import nltk
# from nltk.tokenize import sent_tokenize

# # Download necessary NLTK data
# nltk.download('punkt')

# def clean_text(text):
#     """Removes excessive newlines while preserving structure and removes standalone author mentions."""
#     text = re.sub(r'\n+', '\n', text)  # Replace multiple newlines with a single one
#     text = re.sub(r'\bRick Riordan\b(?!\w)', '', text, flags=re.IGNORECASE)  # Remove only standalone "Rick Riordan"
#     text = re.sub(r'\s+', ' ', text).strip()  # Normalize spaces
#     return text

# def split_into_chunks(text, max_chunk_size=1024):
#     text = clean_text(text)  # Clean text before chunking
#     sentences = sent_tokenize(text)  # Split into sentences
#     chunks = []
#     chunk = ""

#     for sentence in sentences:
#         if len(chunk) + len(sentence) <= max_chunk_size:
#             chunk += sentence + " "
#         else:
#             chunks.append(chunk.strip())
#             chunk = sentence + " "

#     if chunk:
#         chunks.append(chunk.strip())

#     return chunks

# # Apply cleaning and chunking
# cleaned_text = clean_text(book_text)
# chunks = split_into_chunks(cleaned_text)

# print(f"Total chunks: {len(chunks)}")
# print(chunks[:5])  # Print first 5 chunks for verification


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sparshpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total chunks: 508
['', 'index Contents Chapter One ....................................................................................................................................... 1 Chapter Two ........................................................................................................................................ 8 Chapter Three ................................................................................................................................... 14 Chapter Four ..................................................................................................................................... 21 Chapter Five ..................................................................................................................................... 26 Chapter Six ....................................................................................................................................... 30 Chapter Seven ..............................................

In [3]:
# Step 3:
chunks = [chunk for chunk in chunks if chunk.strip()]  # Remove empty or whitespace-only chunks


In [4]:
# Step 4:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")  # Higher accuracy


/Users/sparshpatel/opt/anaconda3/envs/slm_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Step 5:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the same model we used before
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Convert text chunks into embeddings
chunk_embeddings = model.encode(chunks, convert_to_tensor=True)

print("Chunks successfully embedded!")


Chunks successfully embedded!


In [6]:
# Step 6:
from sentence_transformers import SentenceTransformer

# Load the embedding model
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Encode the chunks
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)  # Returns a PyTorch tensor
chunk_embeddings = chunk_embeddings.cpu().numpy()  # Move to CPU and convert to NumPy

# Now rebuild the FAISS index
import faiss
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])  
index.add(chunk_embeddings)  # No more errors!


In [7]:
# Step 7:
import faiss
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(chunk_embeddings)


In [8]:
# Step 8:
from torch.nn.functional import cosine_similarity
import torch

def find_best_chunk(question):
    # Encode the question
    question_embedding = model.encode(question, convert_to_tensor=True)
    
    # Compute similarity with all chunks
    similarities = cosine_similarity(question_embedding, chunk_embeddings)
    
    # Get the index of the most relevant chunk
    best_chunk_idx = torch.argmax(similarities).item()
    
    return chunks[best_chunk_idx]  # Return the most relevant chunk


In [9]:
# Step 9:
from transformers import pipeline

# Load the Question Answering pipeline
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

Device set to use mps:0


In [10]:
# Step 10:
def retrieve_best_chunk(question):
    question_embedding = embedding_model.encode([question], convert_to_numpy=True)
    _, indices = index.search(question_embedding, 1)
    return chunks[indices[0][0]]

def answer_question(question):
    best_chunk = retrieve_best_chunk(question)
    result = qa_pipeline(question=question, context=best_chunk)
    return result["answer"]

question = "Who is the god of underworld??"
print(answer_question(question))


Hades
